In [ ]:
!unzip my_model_signature_fix_3.zip

Archive:  my_model_signature_fix_3.zip
   creating: my_model_signature_fix_3/
  inflating: my_model_signature_fix_3/adapter_model.safetensors  
  inflating: my_model_signature_fix_3/special_tokens_map.json  
  inflating: my_model_signature_fix_3/training_args.bin  
  inflating: my_model_signature_fix_3/tokenizer.json  
  inflating: my_model_signature_fix_3/adapter_config.json  
  inflating: my_model_signature_fix_3/tokenizer.model  
  inflating: my_model_signature_fix_3/tokenizer_config.json  
  inflating: my_model_signature_fix_3/README.md  


In [ ]:
!pip install bitsandbytes transformers accelerate


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.1/76.1 MB 31.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 128.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 87.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 59.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 4.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 43.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 19.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 105.5 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstal

In [ ]:
from peft import PeftConfig, PeftModel
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch

base_model_name = "unsloth/mistral-7b-bnb-4bit"  # Path to your base model
adapter_model_name = "my_model_signature_fix_3"  # Path to your fine-tuned adapter model
max_output_lines = 10                             # Maximum number of output lines to show

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Load base model and adapter
model = AutoModelForCausalLM.from_pretrained(base_model_name, device_map="auto")
model = PeftModel.from_pretrained(model, adapter_model_name)
model = model.bfloat16()
model = model.to(device)

# Load tokenizer
tokenizer = AutoTokenizer.from_pretrained(base_model_name)



tokenizer_config.json:   0%|          | 0.00/1.02k [00:00<?, ?B/s]

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.80M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/438 [00:00<?, ?B/s]

In [ ]:
# Loop to continuously take user input and log input-output pairs
while True:
    input_text = input("Enter your prompt (or type 'exit' to quit): ")
    if input_text.lower() == "exit":
        break

    input_ids = tokenizer(input_text, return_tensors="pt").input_ids.to(device)
    outputs = model.generate(input_ids, max_new_tokens=1000)
    generated_text = tokenizer.decode(outputs[0], skip_special_tokens=True)
    lines = generated_text.split('\n')[:max_output_lines]
    limited_text = '\n'.join(lines)

    print("Generated Output:")
    print(limited_text)
    print("=" * 50)  # Separator for readability

    # Log input and output to a file
    with open("conversation_log.txt", "a", encoding="utf-8") as log_file:
        log_file.write("User Input:\n")
        log_file.write(input_text + "\n\n")
        log_file.write("Model Output:\n")
        log_file.write(limited_text + "\n")
        log_file.write("=" * 50 + "\n\n")


Enter your prompt (or type 'exit' to quit): Write a snort rule that alerts incoming udp packets with the fragbits value set to more fragments.
Generated Output:
Write a snort rule that alerts incoming udp packets with the fragbits value set to more fragments.

alert udp $EXTERNAL_NET any -> $HOME_NET any (msg:'DOS Teardrop attack'; fragbits:M; id:242;sid:666999; rev:2;)
Enter your prompt (or type 'exit' to quit): Write a snort rule that alerts incoming packets to port 22 that have the flow to server and established and the content 'bin/sh'.
Generated Output:
Write a snort rule that alerts incoming packets to port 22 that have the flow to server and established and the content 'bin/sh'.

alert tcp $EXTERNAL_NET any -> $HOME_NET 22 (msg:'EXPLOIT ssh CRC32 overflow /bin/sh'; flow:to_server,established; content:'/bin/sh';sid:607808; rev:9;)
Enter your prompt (or type 'exit' to quit): Write a snort rule that alerts incoming packets to port 3306 that have the flow to server and established a